# Basic Tools for Alhazen

> Simple tools to demonstrate utility and 'agentic' functionality. 

## Available Tools:
1. **EMPCSearchToolSchema**: Execute a query against EPMC, creating a collection and saving expressions, items, and fragments to the database

## Tools under development:
1. Develop queries across external data sources
2. Add all available full text to a collection
3. Extract information from a collection using an LLM-based analysis to create Notes 
4. Filter a collection by an LLM-based analysis by tagging fragments with Notes
5. Report on the state of the database in terms of numbers of collections, expressions, items, and fragments
6. Prepare a report over a core research question by collecting a number of notes and synthesizing them into a report

In [ ]:
#| default_exp tools.basic

In [ ]:
#| export

import os

from alhazen.core import OllamaRunner, PromptTemplateRegistry, get_langchain_llm, get_cached_gguf, \
    get_langchain_embeddings, GGUF_LOOKUP_URL, MODEL_TYPE
from alhazen.utils.airtableUtils import AirtableUtils
from alhazen.utils.searchEngineUtils import ESearchQuery, EuroPMCQuery
from alhazen.utils.langchain_utils import suppress_stdout_stderr
from alhazen.utils.output_parsers import JsonEnclosedByTextOutputParser
from alhazen.utils.airtableUtils import AirtableUtils
from alhazen.utils.searchEngineUtils import ESearchQuery, EuroPMCQuery
from alhazen.utils.queryTranslator import QueryTranslator, QueryType
from alhazen.schema_sqla import ScientificKnowledgeCollection, \
    ScientificKnowledgeExpression, ScientificKnowledgeCollectionHasMembers, \
    ScientificKnowledgeItem, ScientificKnowledgeExpressionHasRepresentation, \
    ScientificKnowledgeFragment, ScientificKnowledgeItemHasPart, \
    InformationResource, Note, NoteIsAbout
from alhazen.utils.ceifns_db import QuerySpec, Ceifns_LiteratureDb
from alhazen.utils.jats_text_extractor import NxmlDoc


from langchain.tools import StructuredTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.llms import Ollama
from langchain.utilities import SerpAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field

import pandas as pd

from sqlalchemy import create_engine, exists, func
from sqlalchemy.orm import sessionmaker

from typing import Optional, Type




In [ ]:
#| export
# NOTE - Use LangChain's SQL_DATABASE TOOL AS A MODEL 
# https://github.com/langchain-ai/langchain/blob/535db72607c4ae308566ede4af65295967bb33a8/libs/community/langchain_community/tools/sql_database/tool.py

class AlhazenToolMixin(BaseModel):
    '''Base tool for interacting with an Alhazen CEIFNS (pron. 'SAI-FiNS') database 
    (CEIFNS = Collection-Expression-Item-Fragment-Note-Summary).'''

    db: Ceifns_LiteratureDb = Field(exclude=True)
    ollr : OllamaRunner = Field(exclude=True)
    llm : Ollama = Field(exclude=True)
    llm_model : str = Field(default='mixtral')

    class Config:
        """Configuration for this pydantic object."""
        arbitrary_types_allowed = True

In [2]:
#| export

class EMPCSearchToolSchema(BaseModel):
    db: Ceifns_LiteratureDb = Field(exclude=True, description="The CEIFNS Database we will be populating")
    query: str = Field(description="should be a search query")
    name: str = Field(description="should be the name of the collection we will add papers to")

class EMPCSearchTool(BaseTool): 
    name = "epmc_search"
    description = "useful for when you need to search for biomedical scientific papers"
    args_schema: Type[EMPCSearchToolSchema] = EMPCSearchToolSchema

    def _run(
        self,
        name: str,
        query: str,
        date_query: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        """Use the tool."""
        if os.environ.get('LOCAL_FILE_PATH') is None: 
            raise Exception('Where are you storing your local literature database?')
        loc = os.environ['LOCAL_FILE_PATH']
        if os.environ.get('DATABASE_NAME') is None: 
            raise Exception('Which database do you want to use for this application?')
        db_name = os.environ['DATABASE_NAME']

        try: 
            
            r = self.db.session.query(func.max(ScientificKnowledgeCollection.id)).first()
            max_id = r[0]
            if max_id is None:
                c_id = '0'
            else:
                c_id = str(int(float(max_id)) + 1)

            cdf1 = pd.DataFrame([{'ID': c_id, 'NAME': name, 'QUERY': query}])        
            qs1 = QuerySpec(db_name, 'ID', 'QUERY', 'NAME', {}, ['TITLE','ABSTRACT'])
            qt1 = QueryTranslator(cdf1.sort_values('ID'), 'ID', 'QUERY', 'NAME')

            if(date_query is not None):
                cdf2 = pd.DataFrame([{'ID': None, 'NAME': None, 'QUERY': date_query}])        
                qs2 = QuerySpec(db_name, 'ID', 'QUERY', 'NAME', {}, ['FIRST_PDATE'])
                qt2 = QueryTranslator(cdf2.sort_values('ID'), 'ID', 'QUERY', 'NAME')
                self.db.add_corpus_from_epmc(qt1, qt2, sections=qs1.sections, sections2=qs2.sections, page_size=100)
            else: 
                self.db.add_corpus_from_epmc(qt1, None, sections=qs1.sections, page_size=100)

            r2 = self.db.session.query(func.count(ScientificKnowledgeExpression.id)) \
                    .filter(ScientificKnowledgeCollection.id == ScientificKnowledgeCollectionHasMembers.ScientificKnowledgeCollection_id) \
                    .filter(ScientificKnowledgeCollectionHasMembers.has_members_id == ScientificKnowledgeExpression.id) \
                    .filter(ScientificKnowledgeCollection.id == str(c_id)).first()
            n_papers_added = r2[0]

        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'action': 'Final Answer', 
                'action_input': 'Successfully constructed a collection called `{}` containing {} papers by querying the European PMC Database.'.format(name, n_papers_added)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("epmc_search does not support async")

In [ ]:
os.environ['LOCAL_FILE_PATH'] = '/Users/gburns/alhazen/'
os.environ['DATABASE_NAME'] = 'temp'

t = EMPCSearchTool()
t._run(name='Stuff', 
       query='Cryoelectron Tomography | Cryo Electron Tomography | Cryo-Electron Tomography | Cryo-ET | CryoE')

In [ ]:
db_name = os.environ['DB_NAME']
loc = os.environ['LLMS_TEMP_DIR']

db = Ceifns_LiteratureDb(loc, db_name)
if db.session is None:
    session_class = sessionmaker(bind=db.engine)
    db.session = session_class()

r2 = db.session.query(func.count(ScientificKnowledgeExpression.id)) \
                    .filter(ScientificKnowledgeCollection.id == ScientificKnowledgeCollectionHasMembers.ScientificKnowledgeCollection_id) \
                    .filter(ScientificKnowledgeCollectionHasMembers.has_members_id == ScientificKnowledgeExpression.id) \
                    .filter(ScientificKnowledgeCollection.id == str(0.0))
print(str(r2.first()[0]))



1853
